In [3]:
import re



In [4]:
CONGESTION_KEYWORDS = [
    "busy", "crowded", "congestion", "wait", "full", "available", "chargers", "charging station"
]

COST_KEYWORDS = [
    "cost", "price", "cheap", "cheaper", "expense", "compare", "km", "trip", "save"
]

HELP_KEYWORDS = [
    "help", "what can you do ?", "commands", "how does this work", "what questions"
]


In [5]:
def detect_intent(user_input):
    text = user_input.lower()
    
    # Check for congestion intent
    for word in CONGESTION_KEYWORDS:
        if word in text:
            return "get_congestion"
    
    # Check for cost comparison intent
    for word in COST_KEYWORDS:
        if word in text:
            return "compare_cost"
    
    # Check for help intent
    for word in HELP_KEYWORDS:
        if word in text:
            return "help"
    
    # If nothing matches
    return "unknown"


In [6]:
def generate_response(intent):
    if intent == "get_congestion":
        return "This looks like a congestion query. Later, this will be connected to the congestion prediction model."
    
    elif intent == "compare_cost":
        return "This looks like a cost comparison query. Later, this will calculate EV vs petrol trip cost."
    
    elif intent == "help":
        return "I can help you check charging congestion or compare EV and petrol trip costs."
    
    else:
        return "Sorry, I could not understand your question. Please try asking about charging congestion or trip cost."


In [10]:
while True:
    user_query = input("Ask a question (type 'exit' to stop): ")
    
    if user_query.lower() == "exit":
        print("Demo ended.")
        break
    
    detected_intent = detect_intent(user_query)
    response = generate_response(detected_intent)
    
    print("Detected Intent:", detected_intent)
    print("System Response:", response)
    print("-" * 50)


Detected Intent: unknown
System Response: Sorry, I could not understand your question. Please try asking about charging congestion or trip cost.
--------------------------------------------------
Detected Intent: compare_cost
System Response: This looks like a cost comparison query. Later, this will calculate EV vs petrol trip cost.
--------------------------------------------------
Detected Intent: compare_cost
System Response: This looks like a cost comparison query. Later, this will calculate EV vs petrol trip cost.
--------------------------------------------------
Detected Intent: unknown
System Response: Sorry, I could not understand your question. Please try asking about charging congestion or trip cost.
--------------------------------------------------
Detected Intent: help
System Response: I can help you check charging congestion or compare EV and petrol trip costs.
--------------------------------------------------
Detected Intent: help
System Response: I can help you check 

: 

: 

In [7]:
test_queries = [
    "how busy is the burwood charger",
    "compare ev and petrol cost for 150 km",
    "what can you do",
    "is there a wait time at melbourne central chargers",
    "how much does an ev trip cost for 100 km",
    "random unrelated question"
]

for q in test_queries:
    intent = detect_intent(q)
    print(f"Query: {q}")
    print(f"Detected Intent: {intent}")
    print("-" * 40)


Query: how busy is the burwood charger
Detected Intent: get_congestion
----------------------------------------
Query: compare ev and petrol cost for 150 km
Detected Intent: compare_cost
----------------------------------------
Query: what can you do
Detected Intent: unknown
----------------------------------------
Query: is there a wait time at melbourne central chargers
Detected Intent: get_congestion
----------------------------------------
Query: how much does an ev trip cost for 100 km
Detected Intent: compare_cost
----------------------------------------
Query: random unrelated question
Detected Intent: unknown
----------------------------------------
